In [3]:
import pandapower as pp
import pandapower.timeseries as ts
import pandapower.control as control
from pandapower.timeseries import DFData
import pandas as pd
import numpy as np

# Create an empty network
net = pp.create_empty_network()

# Add buses
bus1 = pp.create_bus(net, vn_kv=110)
bus2 = pp.create_bus(net, vn_kv=110)
bus3 = pp.create_bus(net, vn_kv=110)

# Add lines
pp.create_line(net, bus1, bus2, length_km=10, std_type="NAYY 4x50 SE")
pp.create_line(net, bus2, bus3, length_km=10, std_type="NAYY 4x50 SE")

# Add loads
pp.create_load(net, bus3, p_mw=100)

# Add a battery storage
pp.create_storage(net, bus2, p_mw=0, max_e_mwh=1, soc_percent=50, min_e_mwh=0.1, max_p_mw=0.5, min_p_mw=-0.5)

# Set the slack bus
pp.create_ext_grid(net, bus1)

# Create timeseries data
time_steps = pd.date_range('2023-01-01', periods=96, freq='15T')  # 24 hours with 15-minute intervals
load_profile = pd.DataFrame({'p_mw': 100 + 10 * np.sin(np.linspace(0, 3.14*2, 96))}, index=time_steps)

# Create a DataFrame to hold the timeseries data
ds_load = DFData(load_profile)

# Create a control strategy for the load
load_idx = net.load.index[0]
const_load = control.ConstControl(net, element='load', variable='p_mw', element_index=load_idx, data_source=ds_load, profile_name='p_mw')

# Debug: Check load control setup
print("Load control setup:")
print(const_load)

# Add an OutputWriter to store the results
ow = ts.OutputWriter(net, output_path="./", output_file_type=".csv")
ow.log_variable('res_bus', 'vm_pu')
ow.log_variable('res_line', 'loading_percent')
ow.log_variable('res_storage', 'soc_percent')

# Time step settings
n_ts = len(load_profile)  # Number of time steps

# Run timeseries simulation
try:
    pp.timeseries.run_timeseries(net, time_steps=range(n_ts))
except KeyError as e:
    print(f"KeyError: {e}. Please check the indices and timeseries data sources.")
except Exception as e:
    print(f"An error occurred: {e}")

# Debug: Check network state after timeseries simulation
print("Network state after timeseries simulation:")
print(net)

# Collect results
results = []
for t in range(n_ts):
    time_step_results = {
        'time': load_profile.index[t],
        'bus': net.res_bus['vm_pu'].values if 'vm_pu' in net.res_bus.columns else [],
        'line': net.res_line['loading_percent'].values if 'loading_percent' in net.res_line.columns else []
    }
    
    if 'soc_percent' in net.res_storage.columns:
        time_step_results['storage'] = net.res_storage['soc_percent'].values
    else:
        time_step_results['storage'] = None

    results.append(time_step_results)

# Print results for the first timestep
print(results[0])

# Print all results for debugging
print(results)


Load control setup:
ConstControl [load.p_mw]


  1%|          | 1/96 [00:00<00:00, 400.14it/s]

KeyError: 0. Please check the indices and timeseries data sources.
Network state after timeseries simulation:
This pandapower network includes the following parameter tables:
   - bus (3 elements)
   - load (1 element)
   - storage (1 element)
   - ext_grid (1 element)
   - line (2 elements)
   - controller (1 element)
   - output_writer (1 element)
{'time': Timestamp('2023-01-01 00:00:00', freq='15T'), 'bus': array([], dtype=float64), 'line': array([], dtype=float64), 'storage': None}
[{'time': Timestamp('2023-01-01 00:00:00', freq='15T'), 'bus': array([], dtype=float64), 'line': array([], dtype=float64), 'storage': None}, {'time': Timestamp('2023-01-01 00:15:00', freq='15T'), 'bus': array([], dtype=float64), 'line': array([], dtype=float64), 'storage': None}, {'time': Timestamp('2023-01-01 00:30:00', freq='15T'), 'bus': array([], dtype=float64), 'line': array([], dtype=float64), 'storage': None}, {'time': Timestamp('2023-01-01 00:45:00', freq='15T'), 'bus': array([], dtype=float64), 